In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('/home/fayari/TEST/data_collection/data.csv')
df.head()

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha
0,0.984234,0.967931,0.760961,0.595673,0.015892,0.402452,0.402452,81.357114,1137.064781,0.370253,0.946187,0.150275,408.015229,1836.068532,0.154730,0.151289,5.695816,3.743539,NaN,NaN
1,0.808703,0.502863,0.962999,0.754977,0.212324,0.116173,0.116173,69.669734,2695.217813,0.364243,0.910936,0.347431,408.188358,1836.847609,0.349206,0.347828,2.122562,0.747097,NaN,NaN
2,0.266592,0.973465,0.748812,0.580608,1.322036,0.400463,0.400463,17.550390,1081.897319,0.672173,0.896924,0.953452,408.009100,1836.040949,0.953519,0.953467,0.929959,0.610609,NaN,NaN
3,0.830782,0.963485,0.899421,0.270474,0.185388,0.039302,0.039302,44.256621,81.149965,0.775990,0.869940,0.716221,407.897906,1835.540575,0.716701,0.716328,1.382759,1.340499,NaN,NaN
4,0.600291,0.839312,0.707039,0.652968,0.510340,0.652287,0.652287,24.480144,932.892398,0.777209,0.878946,0.910105,407.992544,1835.966446,0.910238,0.910135,0.990440,0.689159,NaN,NaN


In [4]:
df.columns

Index(['Tg_scat', 'Tg_abs', 'Ta_abs', 'SSA', 'GOD', 'AOD', 'AODS', 'SZA', 'Z',
       'R_scence', 'g1', 'Cos(SZA)', 'mu_g', 'mu_a', 'muprime_g', 'muprime_a',
       'mprime_g', 'mprime_a', 'BOA_RT', 'alpha'],
      dtype='object')

In [5]:
df['AODS']

0         0.402452
1         0.116173
2         0.400463
3         0.039302
4         0.652287
            ...   
377995    0.214810
377996    0.608888
377997    0.694040
377998    0.103640
377999    0.587625
Name: AODS, Length: 378000, dtype: float64

In [6]:
df['AOD']

0         0.402452
1         0.116173
2         0.400463
3         0.039302
4         0.652287
            ...   
377995    0.214810
377996    0.608888
377997    0.694040
377998    0.103640
377999    0.587625
Name: AOD, Length: 378000, dtype: float64

In [9]:
import math
1023.93019083937*math.cos(math.radians(12.4115211))

999.9999996158948

In [18]:
import math
def EXTRACT_TOA(path,SZA):
    with open(path, 'r') as file:
        for line in file:  # ← ici, on lit ligne par ligne
            if line.strip().startswith("phase.band0.spectral.TOA:"):
                valeur = float(line.strip().split(":")[1])
                return valeur*math.cos(math.radians(SZA))
    return None  

In [19]:
print(EXTRACT_TOA('/home/fayari/DART1424/user_data/simulations/Analytic/output/simulation.properties.txt',12.4115211))

999.9999996158948


In [20]:
def edit_phase_scn(filepath, parameter, new_value):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    with open(filepath, 'w') as file:
        for line in lines:
            if line.strip().startswith(parameter + "="):
                file.write(f"{parameter}={new_value}\n")
            else:
                file.write(line)

In [22]:
edit_phase_scn('/home/fayari/DART1424/user_data/simulations/Analytic/output/phase.scn', 'scene.lights.sky.color', 160)

In [23]:
def EXTRACT_TOA(path,SZA):
    with open(path, 'r') as file:
        for line in file:  # ← ici, on lit ligne par ligne
            if line.strip().startswith("phase.band0.spectral.TOA:"):
                valeur = float(line.strip().split(":")[1])
                return valeur*math.cos(math.radians(SZA))
    return None  

In [24]:
EXTRACT_TOA('/home/fayari/DART1424/user_data/simulations/Analytic/output/simulation.properties.txt',12.4115211)

999.9999996158948

In [25]:
def EXTRACT_E_direct(path,SZA):
    with open(path, 'r') as file:
        for line in file:  # ← ici, on lit ligne par ligne
            if line.strip().startswith("scene.lights.sunlight.color="):
                valeur = float(line.strip().split("=")[1])
                return valeur*math.cos(math.radians(SZA))
    return None  

In [26]:
EXTRACT_E_direct('/home/fayari/DART1424/user_data/simulations/Analytic/output/phase.scn',12.4115211)

325.3737440820283

In [4]:
def calculate_alpha(thetha,z,Tg_abs,Tg_scat,optical_depth,albedo,n):   
    def EXTRACT_BOA_RT(n):
        with open(f'/home/fayari/DART1424/user_data/simulations/RTM_reference{n}/output/dart.txt','r') as file :
            boa_values = []
            for line in file :
                if "BOA Total" in line:
                    value = float(line.split("*")[-1].strip())  
                    boa_values.append(value)
        return boa_values[-1]  
    def muprime(z,h,µ):
        RAYON_TERRESTRE=6371
        eta = (RAYON_TERRESTRE*1000 + z) / h
        root = (eta*µ)**2  + 2 * eta + 1
        sum = (root)**0.5 - eta * µ
        if sum > 0 :
            return 1/sum
        return 1 
    BOA_RT=EXTRACT_BOA_RT(n)
    E_TOA=1000
    Ha=2000
    Hg=9000
    mu_a=(3671*1000+z)/Ha
    mu_g=(3671*1000+z)/Hg
    angle_rad = math.radians(thetha)
    µ=math.cos(angle_rad)  
    Y_a=muprime(z,Ha,µ)
    Y_g=muprime(z,Hg,µ)
    Ma=math.exp(-z/Ha)/Y_a
    Mg=math.exp(-z/Hg)/Y_g
    delta_a_scat=optical_depth*albedo
    delta_g_scat=-math.log(Tg_scat)
    Ta_abs=math.exp(-optical_depth*(1-albedo))
    numerator= (E_TOA/BOA_RT)*(Tg_abs**(Mg))*(Ta_abs**(Ma))-1
    denominator=(delta_g_scat * Mg) + (delta_a_scat * Ma) / 3
    alpha= numerator / denominator
    return alpha

In [1]:
from functions import *

In [9]:
SZA=get_SZA('/home/fayari/DART1425/user_data/simulations/RTM_reference0/input/directions.xml')
z=get_altitude('/home/fayari/DART1425/user_data/simulations/RTM_reference0/input/maket.xml')
reflectane=get_reflectance('/home/fayari/DART1425/user_data/simulations/RTM_reference0/input/coeff_diff.xml')
tg_scat, tg_abs, optical_depth, albedo, a=atmosphere_param('/home/fayari/DART1425/user_data/simulations/RTM_reference0/output/atmosphere.nc')

In [10]:
calculate_alpha(SZA,z,tg_abs,tg_scat,optical_depth,albedo,0)

0.1783139950837795

In [11]:
calculate_BOA_TOTAL(0.1783139950837795,SZA,z,tg_abs,tg_scat,optical_depth,albedo,999.999918)

908.6951232599898

In [12]:
908.6951977729962/math.cos(math.radians(SZA))

930.4404472705246

In [ ]:
from netCDF4 import Dataset
def atmosphere_param(atmosphere_path):
    with Dataset(atmosphere_path, "r") as nc_file:
        tg_scat = nc_file.groups['Transmittance']['All_gas_transmittance'][0][0]
        tg_abs = np.prod(nc_file.groups['Transmittance']['All_gas_transmittance'][0][1:15])
        optical_depth = nc_file.groups['Transmittance']['Aerosols optical_depth'][0][0]
        albedo = nc_file.groups['Transmittance']['Aerosol_single_scattering albedo'][0][0]
        a = nc_file.groups['Henyey Greenstein']['a'][0][0]
        g1 = nc_file.groups['Henyey Greenstein']['g1'][0][0]
        g2 = nc_file.groups['Henyey Greenstein']['g2'][0][0]
    return tg_scat, tg_abs, optical_depth, albedo, a

In [1]:
from netCDF4 import Dataset
nc_file=Dataset('/home/fayari/DART1425/user_data/simulations/Analytic/output/atmosphere.nc','r')
nc_file

<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Already_processed_by_phase: 1
    With_or_without_database: 0
    Height_of_sensor: 3000
    Aerosol_scale_factor: 1000.0
    Total_aerosols_optical_depth: 0.202
    Aerosol_albedo: 0.947
    Henyey_Greenstein_g1: 0.79
    Henyey_Greenstein_g2: 0.4
    Henyey_Greenstein_a: 0.95
    Gas_optical_depth_scattering: 0.049
    Gas_scale_factor: 8400.0
    Transmittance_of_gases_absorb: 0.75
    dimensions(sizes): Altitude_for_temperature_pressure(33), Row_altitude_temperature_pressure(3)
    variables(dimensions): 
    groups: Vertical distribution

In [ ]:
from netCDF4 import Dataset
def atmosphere_param(atmosphere_path):
    with Dataset(atmosphere_path, "r") as nc_file:
        GOD = nc_file.getncattr('Gas_optical_depth_scattering')
        tg_scat=math.exp(-GOD)
        tg_abs = nc_file.getncattr('Transmittance_of_gases_absorb')
        optical_depth = nc_file.getncattr('Total_aerosols_optical_depth')
        albedo = nc_file.getncattr('Aerosol_albedo')
        a = nc_file.getncattr('Henyey_Greenstein_a')
        g1 = nc_file.getncattr('Henyey_Greenstein_g1')
        g2 = nc_file.getncattr('Henyey_Greenstein_g2')
    return tg_scat, tg_abs, optical_depth, albedo, a

0.79

In [8]:
import pandas as pd
d=pd.read_csv('/home/fayari/TEST/data_collection/dataset.csv')
d.head()

,lambda,Tg_scat,Tg_abs,albedo,aerosol_optical_depth,Zenith_angle,Z,Reflectance,a,g1,g2,BOA_RT,alpha
0,0.4,0.6974,0.996201,0.952949,0.44457,0.0,0.0,0.1,1.0,0.899439,0.0,828.253839,0.354606
1,0.4,0.6974,0.996201,0.952949,0.44457,0.0,500.0,0.1,1.0,0.899439,0.0,844.606610,0.356321
2,0.4,0.6974,0.996201,0.952949,0.44457,0.0,1000.0,0.1,1.0,0.899439,0.0,858.435498,0.358480
3,0.4,0.6974,0.996201,0.952949,0.44457,0.0,1500.0,0.1,1.0,0.899439,0.0,870.475820,0.360011
4,0.4,0.6974,0.996201,0.952949,0.44457,0.0,2000.0,0.1,1.0,0.899439,0.0,881.099162,0.360666


In [11]:
import math 
-math.log(0.6974)

0.36039614474058623

In [12]:
atmosphere_nc='/home/fayari/DART1425/user_data/simulations/Analytic/output/atmosphere.nc'
atmosphere_xml='/home/fayari/DART1425/user_data/simulations/Analytic/input/atmosphere.xml'

In [76]:
import xml.etree.ElementTree as ET
import numpy as np 
def atmosphere_param(atmosphere_nc,atmosphere_xml):
    tree = ET.parse(atmosphere_xml)  # Remplace 'votre_fichier.xml' par le nom réel
    root = tree.getroot()
    l=[]
    # Trouver l'élément <IsAtmosphere> et lire l'attribut 'typeOfAtmosphere'
    is_atmosphere_elem = root.find('.//IsAtmosphere')  # Recherche récursive
    type_value = int(is_atmosphere_elem.get('typeOfAtmosphere'))
    if type_value == 1 :

        with Dataset(atmosphere_nc, "r") as nc_file:
            spectral_number=nc_file.groups['Transmittance']['All_gas_transmittance'].shape[0]
            for i in range(spectral_number):
                tg_scat = nc_file.groups['Transmittance']['All_gas_transmittance'][i][0]
                tg_abs = np.prod(nc_file.groups['Transmittance']['All_gas_transmittance'][i][1:15])
                optical_depth = nc_file.groups['Transmittance']['Aerosols optical_depth'][i][0]
                albedo = nc_file.groups['Transmittance']['Aerosol_single_scattering albedo'][i][0]
                a = nc_file.groups['Henyey Greenstein']['a'][i][0]
                g1 = nc_file.groups['Henyey Greenstein']['g1'][i][0]
                g2 = nc_file.groups['Henyey Greenstein']['g2'][i][0]
                l.append([tg_scat, tg_abs, optical_depth, albedo, a, g1])
            return l
    else :

        with Dataset(atmosphere_nc, "r") as nc_file:
            GOD = nc_file.getncattr('Gas_optical_depth_scattering')
            tg_scat=math.exp(-GOD)
            tg_abs = nc_file.getncattr('Transmittance_of_gases_absorb')
            optical_depth = nc_file.getncattr('Total_aerosols_optical_depth')
            albedo = nc_file.getncattr('Aerosol_albedo')
            a = nc_file.getncattr('Henyey_Greenstein_a')
            g1 = nc_file.getncattr('Henyey_Greenstein_g1')
            g2 = nc_file.getncattr('Henyey_Greenstein_g2')
            l.append([tg_scat, tg_abs, optical_depth, albedo, a, g1])
            return l

In [83]:
tg_scat, tg_abs, optical_depth, albedo, a, g1 =atmosphere_param(atmosphere_nc,atmosphere_xml)[0]

In [84]:
tg_scat, tg_abs, optical_depth, albedo, a, g1

(0.6974,
 0.9962010699299999,
 0.444569827989,
 0.952949464992,
 1.0,
 0.8994393507592)

In [97]:
def calculate_BOA_TOTAL(SZA,Z,alpha,l,E_TOA):   
    def muprime(z,h,µ):
        RAYON_TERRESTRE=6371
        eta = (RAYON_TERRESTRE*1000 + z) / h
        root = (eta*µ)**2  + 2 * eta + 1
        sum = (root)**0.5 - eta * µ
        if sum > 0 :
            return 1/sum
        return 1 
    BOA=[]
    for i in l :
        tg_scat, tg_abs, optical_depth, albedo, a, g1 = i
        Ha=2000
        Hg=9000
        mu_a=(3671*1000+Z)/Ha
        mu_g=(3671*1000+Z)/Hg
        angle_rad = math.radians(SZA)
        µ=math.cos(angle_rad)  
        Y_a=muprime(Z,Ha,µ)
        Y_g=muprime(Z,Hg,µ)
        Ma=math.exp(-Z/Ha)/Y_a
        Mg=math.exp(-Z/Hg)/Y_g
        delta_a_scat=optical_depth*albedo
        delta_g_scat=-math.log(tg_scat)
        Ta_abs=math.exp(-optical_depth*(1-albedo))
        numerator= E_TOA*(tg_abs**Mg)*(Ta_abs**Ma)
        denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*delta_a_scat)*Ma
        BOA_i= numerator / denominator
        BOA.append(BOA_i)
    return BOA


In [91]:
def EXTRACT_E_direct(path,SZA):
    with open(path, 'r') as file:
        for line in file:  # ← ici, on lit ligne par ligne
            if line.strip().startswith("scene.lights.sunlight.color="):
                valeur_str = line.strip().split("=")[1]
                return [ float(val)*math.cos(math.radians(SZA)) for val in valeur_str.split() ]
    return None 

In [104]:
E_direct=EXTRACT_E_direct('/home/fayari/DART1425/user_data/simulations/Analytic/output/phase.scn',0)
E_direct

[445.405, 662.163, 599.33]

In [105]:
BOA_TOTAL=calculate_BOA_TOTAL(0,0,0.354606,atmosphere_param(atmosphere_nc,atmosphere_xml),1000) 
BOA_TOTAL

[828.2538704116978, 883.5052019768608, 890.0740905044623]

In [106]:
atmosphere_param(atmosphere_nc,atmosphere_xml)

[[0.6974,
  0.9962010699299999,
  0.444569827989,
  0.952949464992,
  1.0,
  0.8994393507592],
 [0.9342,
  0.9481034047393616,
  0.290887203401,
  0.946995768297,
  1.0,
  0.9019767360860714],
 [0.8663999999999998,
  0.9882108890279999,
  0.356674943939,
  0.950786676928,
  1.0,
  0.8828255966164]]

In [107]:
if len(BOA_TOTAL)==len(E_direct) :
    E_diffus=[i-j for i,j in zip(BOA_TOTAL,E_direct)]
else :
    E_diffus=[BOA_TOTAL[0]-i for i in E_direct]

E_diffus

[382.8488704116978, 221.3422019768608, 290.7440905044623]

In [24]:
import re

# Chemin vers le fichier .scn
file_path = "/home/fayari/DART1425/user_data/simulations/tes__reflectance/output/maket.scn"

# Regex pour capturer la ligne recherchée
pattern_id = re.compile(r"scene\.objects\.object[\d_\-]+\.dartNameId=fg0")
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()
for i, line in enumerate(lines):
    if pattern_id.search(line):
        break
target_line = lines[i + 4].strip()
value = target_line.split('=', 1)[1]
print(value)
pattern_ref = re.compile(fr"scene\.materials\.{re.escape(value)}\.ref=")
for i, line in enumerate(lines):
    if pattern_ref.search(line):
        break
ref = line.split('=', 1)[1]
ref = ref.strip()
print(ref)
phase_path='phase.scn'
pattern_value=re.compile(fr"scene\.materials\.{re.escape(ref)}\.kd=(.+)")
with open(phase_path, "r", encoding="utf-8") as file:
    lines = file.readlines()
for i, line in enumerate(lines):
    if pattern_value.search(line):
        reflectance_value = line.split('=', 1)[1]
        break
print(reflectance_value)

_MKT2_
lambertian0
0.09378698428



In [41]:
import re
def get_reflectance(macket_scn,phase_scn):
    pattern_id = re.compile(r"scene\.objects\.object[\d_\-]+\.dartNameId=fg0")
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if pattern_id.search(line):
            break
    target_line = lines[i + 4].strip()
    value = target_line.split('=', 1)[1]
    pattern_ref = re.compile(fr"scene\.materials\.{re.escape(value)}\.ref=")
    for i, line in enumerate(lines):
        if pattern_ref.search(line):
            break
    ref = line.split('=', 1)[1]
    ref = ref.strip()
    pattern_value=re.compile(fr"scene\.materials\.{re.escape(ref)}\.kd=(.+)")
    with open(phase_scn, "r", encoding="utf-8") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if pattern_value.search(line):
            values = line.split('=', 1)[1]
            reflectance_values=values.split(' ')
            reflectance_values=[float(i.strip()) for i in reflectance_values]
            break
    return reflectance_values

In [46]:
macket = '/home/fayari/DART1425/user_data/simulations/Analytic/output/macket.scn'
phase = '/home/fayari/DART1425/user_data/simulations/Analytic/output/phase.scn'
get_reflectance(macket,phase)

[0.2, 0.3, 0.8, 0.6]

In [49]:
import pandas as pd 
df=pd.read_csv('/home/fayari/TEST/data_collection/dataset.csv')
df.head(20)

,lambda,Tg_scat,Tg_abs,albedo,aerosol_optical_depth,Zenith_angle,Z,Reflectance,a,g1,g2,BOA_RT,alpha
0,0.4,0.6974,0.996201,0.952949,0.44457,0.0,0.0,0.10,1.0,0.899439,0.0,828.253839,0.354606
1,0.4,0.6974,0.996201,0.952949,0.44457,0.0,500.0,0.10,1.0,0.899439,0.0,844.606610,0.356321
2,0.4,0.6974,0.996201,0.952949,0.44457,0.0,1000.0,0.10,1.0,0.899439,0.0,858.435498,0.358480
3,0.4,0.6974,0.996201,0.952949,0.44457,0.0,1500.0,0.10,1.0,0.899439,0.0,870.475820,0.360011
4,0.4,0.6974,0.996201,0.952949,0.44457,0.0,2000.0,0.10,1.0,0.899439,0.0,881.099162,0.360666
5,0.4,0.6974,0.996201,0.952949,0.44457,0.0,2500.0,0.10,1.0,0.899439,0.0,890.472436,0.360749
6,0.4,0.6974,0.996201,0.952949,0.44457,0.0,3000.0,0.10,1.0,0.899439,0.0,898.731111,0.360601
7,0.4,0.6974,0.996201,0.952949,0.44457,0.0,3500.0,0.10,1.0,0.899439,0.0,906.143205,0.359874
8,0.4,0.6974,0.996201,0.952949,0.44457,0.0,4000.0,0.10,1.0,0.899439,0.0,912.785052,0.358880
9,0.4,0.6974,0.996201,0.952949,0.44457,0.0,4500.0,0.10,1.0,0.899439,0.0,918.833059,0.357366


In [50]:
df[df['lambda']==0.65]

,lambda,Tg_scat,Tg_abs,albedo,aerosol_optical_depth,Zenith_angle,Z,Reflectance,a,g1,g2,BOA_RT,alpha
891,0.65,0.952,0.97521,0.944437,0.262874,0.0,0.0,0.10,1.0,0.929435,0.0,938.823797,0.179582
892,0.65,0.952,0.97521,0.944437,0.262874,0.0,500.0,0.10,1.0,0.929435,0.0,944.851014,0.196805
893,0.65,0.952,0.97521,0.944437,0.262874,0.0,1000.0,0.10,1.0,0.929435,0.0,949.634043,0.218290
894,0.65,0.952,0.97521,0.944437,0.262874,0.0,1500.0,0.10,1.0,0.929435,0.0,953.549914,0.242846
895,0.65,0.952,0.97521,0.944437,0.262874,0.0,2000.0,0.10,1.0,0.929435,0.0,956.759901,0.270725
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,0.65,0.952,0.97521,0.944437,0.262874,80.0,3000.0,0.85,1.0,0.929435,0.0,805.691358,0.331201
1778,0.65,0.952,0.97521,0.944437,0.262874,80.0,3500.0,0.85,1.0,0.929435,0.0,816.813325,0.356357
1779,0.65,0.952,0.97521,0.944437,0.262874,80.0,4000.0,0.85,1.0,0.929435,0.0,825.545393,0.388042
1780,0.65,0.952,0.97521,0.944437,0.262874,80.0,4500.0,0.85,1.0,0.929435,0.0,832.688351,0.424325


In [65]:
df.iloc[2]

lambda                      0.400000
Tg_scat                     0.697400
Tg_abs                      0.996201
albedo                      0.952949
aerosol_optical_depth       0.444570
Zenith_angle                0.000000
Z                        1000.000000
Reflectance                 0.100000
a                           1.000000
g1                          0.899439
g2                          0.000000
BOA_RT                    858.435498
alpha                       0.358480
Name: 2, dtype: float64

In [75]:
df.iloc[893]

lambda                      0.650000
Tg_scat                     0.952000
Tg_abs                      0.975210
albedo                      0.944437
aerosol_optical_depth       0.262874
Zenith_angle                0.000000
Z                        1000.000000
Reflectance                 0.100000
a                           1.000000
g1                          0.929435
g2                          0.000000
BOA_RT                    949.634043
alpha                       0.218290
Name: 893, dtype: float64

In [55]:
df.iloc[1795]

lambda                      1.030000
Tg_scat                     0.992400
Tg_abs                      0.997443
albedo                      0.882912
aerosol_optical_depth       0.136019
Zenith_angle                0.000000
Z                        1000.000000
Reflectance                 0.150000
a                           1.000000
g1                          0.942715
g2                          0.000000
BOA_RT                    986.843294
alpha                       0.041607
Name: 1795, dtype: float64

In [ ]:
df.iloc[2730]

lambda                      1.800000
Tg_scat                     0.999200
Tg_abs                      0.025686
albedo                      0.847514
aerosol_optical_depth       0.050031
Zenith_angle                0.000000
Z                        1000.000000
Reflectance                 0.550000
a                           1.000000
g1                          0.890050
g2                          0.000000
BOA_RT                    102.176857
alpha                     -68.064358
Name: 2730, dtype: float64

In [ ]:
import pandas as pd 
df=pd.read_csv('/home/fayari/TEST/data_collection/dataset.csv')
df[(df.Z==0 )& (df.Reflectance == 0.33) & (df.Zenith_angle==10)]

,lambda,Tg_scat,Tg_abs,albedo,aerosol_optical_depth,Zenith_angle,Z,Reflectance,a,g1,g2,BOA_RT,alpha
132,0.40,0.6974,0.996201,0.952949,0.444570,10.0,0.0,0.33,1.0,0.899439,0.0,876.090758,0.222121
1023,0.65,0.9520,0.975210,0.944437,0.262874,10.0,0.0,0.33,1.0,0.929435,0.0,952.258864,0.064449
1914,1.03,0.9924,0.997443,0.882912,0.136019,10.0,0.0,0.33,1.0,0.942715,0.0,982.152941,-0.015762
2805,1.80,0.9992,0.025686,0.847514,0.050031,10.0,0.0,0.33,1.0,0.890050,0.0,23.927739,0.445747


In [106]:
import xml.etree.ElementTree as ET
from netCDF4 import Dataset
def EXTRACT_BOA_RT():
    with open('/home/fayari/DART1425/user_data/simulations/test_seq/sequence/my_seq_0/output/dart.txt','r') as file :
        boa_values = []
        for line in file :
            if "BOA Total" in line:
                value = float(line.split("*")[-1].strip())  
                boa_values.append(value)
    return boa_values[-1]  




    
def calculate_alpha(thetha,z,Tg_abs,Tg_scat,optical_depth,albedo,BOA_RT):   
    def muprime(z,h,µ):
        RAYON_TERRESTRE=6371
        eta = (RAYON_TERRESTRE*1000 + z) / h
        root = (eta*µ)**2  + 2 * eta + 1
        sum = (root)**0.5 - eta * µ
        if sum > 0 :
            return 1/sum
        return 1 
    E_TOA=1000
    Ha=2000
    Hg=9000
    mu_a=(3671*1000+z)/Ha
    mu_g=(3671*1000+z)/Hg
    angle_rad = math.radians(thetha)
    µ=math.cos(angle_rad)  
    Y_a=muprime(z,Ha,µ)
    Y_g=muprime(z,Hg,µ)
    Ma=math.exp(-z/Ha)/Y_a
    Mg=math.exp(-z/Hg)/Y_g
    delta_a_scat=optical_depth*albedo
    delta_g_scat=-math.log(Tg_scat)
    Ta_abs=math.exp(-optical_depth*(1-albedo))
    numerator= (E_TOA/BOA_RT)*(Tg_abs**(Mg))*(Ta_abs**(Ma))-1
    denominator=(delta_g_scat * Mg) + (delta_a_scat * Ma) / 3
    alpha= numerator / denominator
    return alpha

def atmosphere_param(atmosphere_nc,atmosphere_xml):
    tree = ET.parse(atmosphere_xml)  # Remplace 'votre_fichier.xml' par le nom réel
    root = tree.getroot()
    l=[]
    # Trouver l'élément <IsAtmosphere> et lire l'attribut 'typeOfAtmosphere'
    is_atmosphere_elem = root.find('.//IsAtmosphere')  # Recherche récursive
    type_value = int(is_atmosphere_elem.get('typeOfAtmosphere'))
    if type_value == 1 :

        with Dataset(atmosphere_nc, "r") as nc_file:
            spectral_number=nc_file.groups['Transmittance']['All_gas_transmittance'].shape[0]
            for i in range(spectral_number):
                tg_scat = nc_file.groups['Transmittance']['All_gas_transmittance'][i][0]
                tg_abs = np.prod(nc_file.groups['Transmittance']['All_gas_transmittance'][i][1:15])
                optical_depth = nc_file.groups['Transmittance']['Aerosols optical_depth'][i][0]
                albedo = nc_file.groups['Transmittance']['Aerosol_single_scattering albedo'][i][0]
                a = nc_file.groups['Henyey Greenstein']['a'][i][0]
                g1 = nc_file.groups['Henyey Greenstein']['g1'][i][0]
                g2 = nc_file.groups['Henyey Greenstein']['g2'][i][0]
                l.append([tg_scat, tg_abs, optical_depth, albedo, a, g1])
            return l
    else :

        with Dataset(atmosphere_nc, "r") as nc_file:
            GOD = nc_file.getncattr('Gas_optical_depth_scattering')
            tg_scat=math.exp(-GOD)
            tg_abs = nc_file.getncattr('Transmittance_of_gases_absorb')
            optical_depth = nc_file.getncattr('Total_aerosols_optical_depth')
            albedo = nc_file.getncattr('Aerosol_albedo')
            a = nc_file.getncattr('Henyey_Greenstein_a')
            g1 = nc_file.getncattr('Henyey_Greenstein_g1')
            g2 = nc_file.getncattr('Henyey_Greenstein_g2')
            l.append([tg_scat, tg_abs, optical_depth, albedo, a, g1])
            return l


In [108]:
import numpy as np 
atmosphere_param('/home/fayari/DART1425/user_data/simulations/test_seq/sequence/my_seq_0/output/atmosphere.nc','/home/fayari/DART1425/user_data/simulations/test_seq/sequence/my_seq_0/input/atmosphere.xml')

[[0.9521811296985049, 0.75, 0.202, 0.947, 0.95, 0.79]]

In [109]:
import math
calculate_alpha(20,0,0.75,0.9521811296985049,0.202,0.947,695.591442)

0.38779769227912997

In [ ]:
0.018087821155184 0.007475599564188 0.2120211674379 0.2447679981032

In [4]:
import pandas as pd 
df=pd.read_csv('/home/fayari/TEST/data_collection/dataset.csv')
df[(df.Z==4000 )& (df.Reflectance == 0.33) & (df.Zenith_angle==60)]

,lambda,Tg_scat,Tg_abs,albedo,aerosol_optical_depth,Zenith_angle,Z,Reflectance,a,g1,g2,BOA_RT,alpha
635,0.40,0.6974,0.996201,0.952949,0.444570,60.0,4000.0,0.33,1.0,0.899439,0.0,858.535846,0.305520
1526,0.65,0.9520,0.975210,0.944437,0.262874,60.0,4000.0,0.33,1.0,0.929435,0.0,934.003253,0.383486
2417,1.03,0.9924,0.997443,0.882912,0.136019,60.0,4000.0,0.33,1.0,0.942715,0.0,991.757193,0.033623
3308,1.80,0.9992,0.025686,0.847514,0.050031,60.0,4000.0,0.33,1.0,0.890050,0.0,444.057008,-201.995001


In [1]:
python --version
pip show joblib
pip show scikit-learn


SyntaxError: invalid syntax (1333783213.py, line 2)

In [3]:
import joblib
scaler = joblib.load("scaler.pkl")

# Sauvegarder les attributs importants
scaler_params = {
    "mean_": scaler.mean_,
    "scale_": scaler.scale_,
    "var_": scaler.var_,
    "n_samples_seen_": scaler.n_samples_seen_
}



/home/fayari/miniforge3/envs/myptd/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
scaler_params

{'mean_': array([7.94013745e-01, 8.05838119e-01, 7.27448140e-01, 5.47240365e-01,
        2.79803576e-01, 4.37678891e-01, 2.84402275e-01, 4.36144042e+01,
        1.37638197e+03, 4.99860569e-01, 8.40390179e-01, 6.56018409e-01,
        7.08041820e+02, 3.18618819e+03, 6.57467123e-01, 6.56399140e-01,
        2.34860804e+00, 1.80630644e+00]),
 'scale_': array([2.02590893e-01, 1.95269501e-01, 1.93106355e-01, 2.35940539e-01,
        3.52728730e-01, 3.44704526e-01, 3.01406445e-01, 2.52466254e+01,
        1.21847558e+03, 2.88115921e-01, 1.13456576e-01, 2.94399487e-01,
        1.35386176e-01, 6.09237791e-01, 2.92297212e-01, 2.93805424e-01,
        3.70339519e+00, 4.41763650e+00]),
 'var_': array([4.10430701e-02, 3.81301782e-02, 3.72900642e-02, 5.56679378e-02,
        1.24417557e-01, 1.18821210e-01, 9.08458451e-02, 6.37392094e+02,
        1.48468274e+06, 8.30107842e-02, 1.28723947e-02, 8.66710577e-02,
        1.83294166e-02, 3.71170686e-01, 8.54376604e-02, 8.63216274e-02,
        1.37151359e+01, 1

In [8]:
import joblib
import json
import numpy as np

# Charger le scaler
scaler = joblib.load("scaler.pkl")

# Préparer les paramètres sous forme compatible JSON
scaler_params = {
    "mean_": scaler.mean_.tolist(),
    "scale_": scaler.scale_.tolist(),
    "var_": scaler.var_.tolist(),
    "n_samples_seen_": int(scaler.n_samples_seen_)
}

# Sauvegarder dans un fichier JSON
with open("scaler_params.json", "w") as f:
    json.dump(scaler_params, f)


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('my_model.h5')


In [1]:
import xml.etree.ElementTree as ET

# Charger le fichier XML
tree = ET.parse('/home/fayari/DART1425/user_data/simulations/RTM_reference0/input/phase.xml')  # Remplace par le chemin réel
root = tree.getroot()

# Initialiser la liste pour stocker les valeurs
spectral_dart_modes = []

# Parcourir tous les éléments SpectralIntervalsProperties
for elem in root.iter('SpectralIntervalsProperties'):
    val = elem.get('spectralDartMode')
    if val is not None:
        spectral_dart_modes.append(int(val))

print(spectral_dart_modes)

[0, 1, 0]


In [2]:
def get_spectral_mode(phase_path):
    tree = ET.parse(phase_path)  # Remplace par le chemin réel
    root = tree.getroot()

    # Initialiser la liste pour stocker les valeurs
    spectral_dart_modes = []

    # Parcourir tous les éléments SpectralIntervalsProperties
    for elem in root.iter('SpectralIntervalsProperties'):
        val = elem.get('spectralDartMode')
        if val is not None:
            spectral_dart_modes.append(int(val))

    return spectral_dart_modes



In [3]:
phase_path='/home/fayari/DART1425/user_data/simulations/RTM_reference0/input/phase.xml'
a=get_spectral_mode(phase_path)
a

[0, 1, 0]

In [4]:
def EXTRACT_E_direct(path):
    try :
        with open(path, 'r') as file:
            for line in file:  # ← ici, on lit ligne par ligne
                if line.strip().startswith("scene.lights.sky.color="):
                    valeur_str = line.strip().split("=")[1]
                    return [ float(val) for val in valeur_str.split() ]
        return None  
    except :

        raise ValueError("Valeurs de E_Diffus non trouvées.")

In [9]:
c=EXTRACT_E_direct('/home/fayari/DART1425/user_data/simulations/RTM_reference0/output/phase.scn')
c

[6.8928, 1.34725, 5.57167]

In [6]:
a

[0, 1, 0]

In [8]:
b=[7,5,8]
b

[7, 5, 8]

In [10]:
l=[b[i] if a[i]==0 else c[i] for i in range(len(b)) ]
l

[7, 1.34725, 8]

In [20]:
import xml.etree.ElementTree as ET

# Charger le fichier XML
tree = ET.parse('/home/fayari/DART1425/user_data/simulations/RTM_reference0/input/phase.xml')  # Remplace avec ton chemin
root = tree.getroot()

# Trouver la balise AtmosphereRadiativeTransfer
toa_to_boa_value = None
for elem in root.iter('AtmosphereRadiativeTransfer'):
    toa_to_boa_value = elem.get('TOAtoBOA')
    break  # On s'arrête dès qu'on trouve la première

# Affichage du résultat
if toa_to_boa_value is not None:
    print("TOAtoBOA =", int(toa_to_boa_value))
else:
    print("CAN NOT FIND Atmosphere Radiative Transfer Mode")

TOAtoBOA = 0


In [22]:
def get_RTMode(phase):
    tree = ET.parse(phase)  # Remplace avec ton chemin
    root = tree.getroot()

    # Trouver la balise AtmosphereRadiativeTransfer
    try :
        toa_to_boa_value = None
        for elem in root.iter('AtmosphereRadiativeTransfer'):
            toa_to_boa_value = elem.get('TOAtoBOA')
            break  # On s'arrête dès qu'on trouve la première

        # Affichage du résultat
        if toa_to_boa_value is not None:
            return int(toa_to_boa_value)
    except:
        raise ValueError("CAN NOT FIND Atmosphere Radiative Transfer Mode")
        

In [19]:
type(toa_to_boa_value)

str

In [32]:
import re
import math


In [36]:
def EXTRACT_TOA(path, SZA):
    try :
        E_TOA = []
        pattern = re.compile(r"phase\.band(\d+)\.spectral\.TOA:\s*([0-9.+-eE]+)")

        with open(path, 'r') as file:
            for line in file:
                match = pattern.search(line)
                if match:
                    band_index = int(match.group(1))
                    value = float(match.group(2))
                    corrected_value = value
                    E_TOA.append(corrected_value)

        # Retourner les valeurs triées par index de bande
        return E_TOA
    except :

        raise ValueError("Valeurs d'irradiance non trouvées.")

In [37]:
l=EXTRACT_TOA('/home/fayari/DART1425/user_data/simulations/RTM_reference0/output/simulation.properties.txt', 88)

In [38]:
l

[1921.26977816301,
 1840.20275460276,
 290.461254333333,
 243.442309195578,
 199.787083429422,
 162.797013470238,
 131.847675860806,
 110.763277702871,
 92.06710086145,
 77.6183513374968,
 65.0393254378777,
 55.1663670090226,
 1555.2651734882,
 1235.76509753992,
 974.443339604039,
 765.065122106834,
 612.546211593408,
 499.908084317704,
 415.038298408994,
 346.727296298942]